In [14]:
import cortex
from nilearn import image
from nilearn import surface
import os.path as op
import numpy as np
import nibabel as nib
import pandas as pd
import seaborn as sns
import os
import os.path as op

bids_folder = '/mnt_03/ds-dnumrisk' 
plot_folder = op.join(bids_folder, 'plots_and_ims/gradient_stuff')
phenotype_folder = op.join(bids_folder, 'derivatives/phenotype')

subList = [f[4:6] for f in os.listdir(bids_folder) if f[0:4] == 'sub-' and len(f) == 6]


In [ ]:
npc_disp = pd.read_csv(op.join(phenotype_folder, f'withinNdispersion-NPC_grad-2.csv'))
npc_disp.set_index(['subject','group']).sort_index().iloc[35:50] # pick subs with high NPC dispersion
#sns.lmplot(x='subject', y='NPC_dispersion', data=npc_disp, hue = 'group', height=5, aspect=3)   


In [ ]:
# transform all gradient surfaces to fsaverage5, fsaverage and fsnative surface files (.gii)

grad_folder = 'derivatives/gradients'
task = 'magjudge'
source_space = 'fsaverage5' # 

from utils import transform_surfaces
for sub in subList:
    ex_file = op.join(bids_folder,grad_folder,f'sub-{sub}',f'sub-{sub}_ses-1_task-magjudge_space-fsnative_hemi-L_grad1.surf.gii')
    if ~os.path.exists(ex_file):
        fn = op.join(bids_folder,grad_folder,f'sub-{sub}',f'sub-{sub}_g-aligned_space-fsaverag5_n10.npy')
        GMs = np.load(fn)
        for n_grad in range(1,3):
            gm_both = np.split(GMs[n_grad-1],2) # for i, hemi in enumerate(['L', 'R']): --> left first
            for i, hemi in enumerate(['L', 'R']):
                gii_im_datar = nib.gifti.gifti.GiftiDataArray(data=gm_both[i].astype(np.float32))
                gii_im = nib.gifti.gifti.GiftiImage(darrays= [gii_im_datar])

                out_file = op.join(bids_folder,grad_folder,f'sub-{sub}', f'sub-{sub}_ses-1_task-{task}_space-{source_space}_hemi-{hemi}_grad{n_grad}.surf.gii')
                gii_im.to_filename(out_file) 

                fs_hemi = 'lh' if hemi == 'L' else 'rh'
                transform_surfaces(in_file=out_file, fs_hemi=fs_hemi, bids_folder=bids_folder,
                                    source_space= 'fsaverage5', target_space='fsaverage')
                transform_surfaces(in_file=out_file, fs_hemi=fs_hemi, bids_folder=bids_folder,
                                    source_space= 'fsaverage5', target_space=f'sub-{sub}')     # to fsnative !!
    else:
        print(f'Gradient surface.gii files already exist for sub-{sub}')    

250218-13:04:34,907 nipype.interface INFO:
	 stderr 2025-02-18T13:04:34.907049:** DA[0] has coordsys with intent NIFTI_INTENT_NONE (should be NIFTI_INTENT_POINTSET)


/tmp/ipykernel_3212155/986813271.py:8: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~os.path.exists(ex_file):


250218-13:04:36,751 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:
250218-13:04:36,753 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:7.3.2
250218-13:04:36,755 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:
250218-13:04:36,756 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:setenv SUBJECTS_DIR /mnt_03/ds-dnumrisk/derivatives/freesurfer
250218-13:04:36,757 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:cd /home/ubuntu/git/numrisk/numrisk/fmri_analysis/surface
250218-13:04:36,758 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:mri_surf2surf --hemi lh --tval /mnt_03/ds-dnumrisk/derivatives/gradients/sub-21/sub-21_ses-1_task-magjudge_space-fsaverage_hemi-L_grad1.surf.gii --sval /mnt_03/ds-dnumrisk/derivatives/gradients/sub-21/sub-21_ses-1_task-magjudge_space-fsaverage5_hemi-L_grad1.surf.gii --srcsubject fsaverage5 --trgsubject fsaverage 
250218-13:04:36,758 nipype.interface INFO:
	 stdout 2025-02-18T13:04:36.751696:
25

In [ ]:
n_grad = 2
sub = '46'
space = 'fsaverage' 
pyc_subject = space if space == 'fsaverage' else f'{dataset}.sub-{subject:02d}'

surf_L = surface.load_surf_data(op.join(bids_folder, grad_folder,f'sub-{sub}', f'sub-{sub}_ses-1_task-{task}_space-{space}_hemi-L_grad{n_grad}.surf.gii'))
surf_R = surface.load_surf_data(op.join(bids_folder, grad_folder,f'sub-{sub}', f'sub-{sub}_ses-1_task-{task}_space-{space}_hemi-R_grad{n_grad}.surf.gii'))
surf_map_fsnative = np.concatenate([surf_L, surf_R])
surf_map_fsnative = np.array(surf_map_fsnative, dtype=float)

print(surf_map_fsnative.shape)
ds_2 = cortex.Vertex(surf_map_fsnative, pyc_subject)
cortex.webshow(ds_2, port=8000)  